In [ ]:
# this cell was used to find the correct epsg code for the London Boroughs geopackage

import geopandas as gpd
from shapely.geometry import Point
import matplotlib.pyplot as plt

# Load the GeoPackage containing London borough boundaries
geopackage_path = "London_Boroughs.gpkg"
boroughs = gpd.read_file(geopackage_path)

# Reproject to WGS84 (latitude and longitude)
boroughs = boroughs.to_crs(epsg=4326)

# Example coordinates in WGS84
latitude = 51.495979
longitude = -0.134666
point = Point(longitude, latitude)  # Create a Point object

# Plot borough boundaries
fig, ax = plt.subplots(figsize=(10, 10))
boroughs.plot(ax=ax, color='lightgray', edgecolor='black')

# Plot the point representing the coordinates
ax.scatter(point.x, point.y, color='red', label='Coordinates')
ax.annotate('Coordinates', xy=(point.x, point.y), xytext=(point.x + 0.01, point.y + 0.01), fontsize=10, color='red')

# Set plot title and labels
ax.set_title('London Boroughs with Coordinates (WGS84)')
ax.set_xlabel('Longitude')
ax.set_ylabel('Latitude')
ax.legend()

# Show the plot
plt.show()

In [1]:
import sqlite3
import geopandas as gpd
from shapely.geometry import Point
from tqdm import tqdm

# Function to get borough name for a given point
def get_borough(lon, lat, boroughs_gdf, boroughs_sindex):
    point = Point(lon, lat)
    possible_matches_index = list(boroughs_sindex.intersection(point.bounds))
    possible_matches = boroughs_gdf.iloc[possible_matches_index]
    for _, borough in possible_matches.iterrows():
        if borough['geometry'].contains(point):
            return borough['name']  # Assuming 'name' is the column with the borough names
    return None

# Connect to the SQLite database
conn = sqlite3.connect('crime_data.db')
cursor = conn.cursor()

# Load the London boroughs GeoPackage file
boroughs_gdf = gpd.read_file('London_Boroughs.gpkg')
boroughs_gdf = boroughs_gdf.to_crs(epsg=4326)

# Add a new column 'borough' to the 'metropolitan-stop-and-search' table if it doesn't exist
try:
    cursor.execute('ALTER TABLE "metropolitan-stop-and-search" ADD COLUMN borough TEXT')
    conn.commit()
except sqlite3.OperationalError as e:
    if 'duplicate column name' in str(e):
        pass
    else:
        raise

# Fetch distinct years from the data
cursor.execute('SELECT DISTINCT strftime("%Y", Date) AS year FROM "metropolitan-stop-and-search"')
years = cursor.fetchall()

# Process each year separately
for year in years:
    year = year[0]  # Extract year from tuple
    print(f"Processing data for year {year}")
    
    # Fetch rows for the current year
    cursor.execute('SELECT Date, Longitude, Latitude FROM "metropolitan-stop-and-search" WHERE strftime("%Y", Date) = ?', (year,))
    rows = cursor.fetchall()
    
    # Create a spatial index on the boroughs for faster lookups
    boroughs_sindex = boroughs_gdf.sindex
    
    # Prepare a list of updates
    updates = []
    
    # Process each row and prepare the update statement
    for row in tqdm(rows, desc=f"Processing rows for year {year}"):
        date, lon, lat = row
        if lon is not None and lat is not None:
            borough_name = get_borough(lon, lat, boroughs_gdf, boroughs_sindex)
            if borough_name:
                updates.append((borough_name, date))
    
    # Execute the updates in bulk
    cursor.executemany('UPDATE "metropolitan-stop-and-search" SET borough = ? WHERE Date = ?', updates)
    conn.commit()

# Close the connection
conn.close()

Processing data for year 2020


Processing rows for year 2020: 100%|██████████| 317505/317505 [03:24<00:00, 1553.09it/s]


In [ ]:
# This cell:
# 1. Connects to the SQLite database.
# 2. Loads the London boroughs GeoPackage file.
# 3. Adds a new column 'borough' to the 'metropolitan-stop-and-search' table if it doesn't exist.
# 4. Fetches all rows from the 'metropolitan-stop-and-search' table.
# 5. Creates a spatial index on the boroughs for faster lookups.
# 6. Defines a function to determine the borough for a given point using the spatial index.
# 7. Prepares a list of updates.
# 8. Processes each row and prepares the update statement.
# 9. Executes the updates in bulk.
# 10. Commits the changes and closes the connection.


import sqlite3
import geopandas as gpd
from shapely.geometry import Point
from tqdm import tqdm

# Connect to the SQLite database
conn = sqlite3.connect('crime_data.db')
cursor = conn.cursor()

# Load the London boroughs GeoPackage file
boroughs_gdf = gpd.read_file('London_Boroughs.gpkg')
boroughs_gdf = boroughs_gdf.to_crs(epsg=4326)


# Add a new column 'borough' to the 'metropolitan-stop-and-search' table if it doesn't exist
try:
    cursor.execute('ALTER TABLE "metropolitan-stop-and-search" ADD COLUMN borough TEXT')
    conn.commit()
except sqlite3.OperationalError as e:
    if 'duplicate column name' in str(e):
        pass
    else:
        raise

# Fetch all rows from the 'metropolitan-stop-and-search' table
cursor.execute('SELECT Date, Longitude, Latitude FROM "metropolitan-stop-and-search"')
rows = cursor.fetchall()

# Create a spatial index on the boroughs for faster lookups
boroughs_sindex = boroughs_gdf.sindex

def get_borough(lon, lat, boroughs_gdf, boroughs_sindex):
    """
    Function to determine the borough for a given point using spatial index
    """
    point = Point(lon, lat)
    possible_matches_index = list(boroughs_sindex.intersection(point.bounds))
    possible_matches = boroughs_gdf.iloc[possible_matches_index]
    for _, borough in possible_matches.iterrows():
        if borough['geometry'].contains(point):
            return borough['name']  
    return None

# Prepare a list of updates
updates = []
empty_coordinates = 0

# Process each row and prepare the update statement
for row in tqdm(rows, desc="Processing rows"):
    date, lon, lat = row
    if lon is not None and lat is not None:
        borough_name = get_borough(lon, lat, boroughs_gdf, boroughs_sindex)
        if borough_name:
            updates.append((borough_name, date))
        else:
            # print the coordinates for which the borough was not found
            print(f'For {lon, lat}: Borough not found')
    else:
        empty_coordinates += 1

# Print the number of rows with empty coordinates
print(f'Number of rows with empty coordinates: {empty_coordinates}')

# Execute the updates in bulk
cursor.executemany('UPDATE "metropolitan-stop-and-search" SET borough = ? WHERE Date = ?', updates)

# Commit the changes and close the connection
conn.commit()
conn.close()


Processing rows:   0%|          | 257/1671089 [00:00<21:36, 1288.25it/s]

Row with Date 2020-01-01T00:00:00+00:00: Borough found: Hackney
Row with Date 2020-01-01T00:01:00+00:00: Borough found: Westminster
Row with Date 2020-01-01T00:05:00+00:00: Invalid coordinates (Longitude: None, Latitude: None)
Row with Date 2020-01-01T00:05:00+00:00: Invalid coordinates (Longitude: None, Latitude: None)
Row with Date 2020-01-01T00:07:00+00:00: Borough found: Waltham Forest
Row with Date 2020-01-01T00:10:00+00:00: Borough found: Camden
Row with Date 2020-01-01T00:10:00+00:00: Borough found: Camden
Row with Date 2020-01-01T00:15:00+00:00: Borough found: Westminster
Row with Date 2020-01-01T00:15:00+00:00: Borough found: Camden
Row with Date 2020-01-01T00:15:00+00:00: Borough found: Westminster
Row with Date 2020-01-01T00:16:00+00:00: Borough found: Kingston upon Thames
Row with Date 2020-01-01T00:20:00+00:00: Borough found: Westminster
Row with Date 2020-01-01T00:20:00+00:00: Invalid coordinates (Longitude: None, Latitude: None)
Row with Date 2020-01-01T00:20:00+00:00: B

Processing rows:   0%|          | 540/1671089 [00:00<20:26, 1362.15it/s]

Row with Date 2020-01-01T08:00:00+00:00: Borough found: Islington
Row with Date 2020-01-01T08:10:00+00:00: Borough found: Westminster
Row with Date 2020-01-01T08:12:00+00:00: Borough found: Bexley
Row with Date 2020-01-01T08:29:00+00:00: Borough found: Southwark
Row with Date 2020-01-01T08:30:00+00:00: Borough found: Barnet
Row with Date 2020-01-01T08:30:00+00:00: Borough found: Southwark
Row with Date 2020-01-01T08:35:00+00:00: Borough found: Southwark
Row with Date 2020-01-01T08:35:00+00:00: Borough found: Southwark
Row with Date 2020-01-01T08:48:00+00:00: Borough found: Kingston upon Thames
Row with Date 2020-01-01T08:57:00+00:00: Borough found: Southwark
Row with Date 2020-01-01T09:00:00+00:00: Borough found: Southwark
Row with Date 2020-01-01T09:30:00+00:00: Borough found: Kensington and Chelsea
Row with Date 2020-01-01T10:20:00+00:00: Borough found: Kingston upon Thames
Row with Date 2020-01-01T10:27:00+00:00: Borough found: Greenwich
Row with Date 2020-01-01T10:30:00+00:00: Boro

Processing rows:   0%|          | 810/1671089 [00:00<21:05, 1319.75it/s]

...

Processing rows:   0%|          | 5185/1671089 [00:03<19:39, 1412.63it/s]

Row with Date 2020-01-07T15:44:00+00:00: Borough found: Croydon
Row with Date 2020-01-07T15:45:00+00:00: Borough found: Southwark
Row with Date 2020-01-07T15:45:00+00:00: Borough found: Haringey
Row with Date 2020-01-07T15:45:00+00:00: Borough found: Southwark
Row with Date 2020-01-07T15:46:00+00:00: Borough found: Southwark
Row with Date 2020-01-07T15:49:00+00:00: Invalid coordinates (Longitude: None, Latitude: None)
Row with Date 2020-01-07T15:50:00+00:00: Borough found: Ealing
Row with Date 2020-01-07T15:50:00+00:00: Invalid coordinates (Longitude: None, Latitude: None)
Row with Date 2020-01-07T15:50:00+00:00: Borough found: Haringey
Row with Date 2020-01-07T15:52:00+00:00: Invalid coordinates (Longitude: None, Latitude: None)
Row with Date 2020-01-07T15:55:00+00:00: Borough found: Hammersmith and Fulham
Row with Date 2020-01-07T15:55:00+00:00: Borough found: Tower Hamlets
Row with Date 2020-01-07T15:55:00+00:00: Borough found: Southwark
Row with Date 2020-01-07T15:55:00+00:00: Boro

Processing rows:   0%|          | 5490/1671089 [00:03<18:59, 1461.53it/s]

Row with Date 2020-01-07T20:30:00+00:00: Borough found: Hillingdon
Row with Date 2020-01-07T20:30:00+00:00: Borough found: Bromley
Row with Date 2020-01-07T20:30:00+00:00: Borough found: Kensington and Chelsea
Row with Date 2020-01-07T20:30:00+00:00: Borough found: Bromley
Row with Date 2020-01-07T20:30:00+00:00: Borough found: Brent
Row with Date 2020-01-07T20:30:00+00:00: Borough found: Westminster
Row with Date 2020-01-07T20:31:00+00:00: Borough found: Camden
Row with Date 2020-01-07T20:34:00+00:00: Borough found: Ealing
Row with Date 2020-01-07T20:35:00+00:00: Borough found: Bexley
Row with Date 2020-01-07T20:37:00+00:00: Borough found: Wandsworth
Row with Date 2020-01-07T20:38:00+00:00: Borough found: Tower Hamlets
Row with Date 2020-01-07T20:40:00+00:00: Borough found: Hillingdon
Row with Date 2020-01-07T20:40:00+00:00: Invalid coordinates (Longitude: None, Latitude: None)
Row with Date 2020-01-07T20:40:00+00:00: Borough found: Westminster
Row with Date 2020-01-07T20:40:00+00:00:

: 

In [1]:
import sqlite3
import geopandas as gpd
from shapely.geometry import Point
from tqdm import tqdm

# Function to get borough name for a given point
def get_borough(lon, lat, boroughs_gdf, boroughs_sindex):
    point = Point(lon, lat)
    possible_matches_index = list(boroughs_sindex.intersection(point.bounds))
    possible_matches = boroughs_gdf.iloc[possible_matches_index]
    for _, borough in possible_matches.iterrows():
        if borough['geometry'].contains(point):
            return borough['name']  # Assuming 'name' is the column with the borough names
    return None

# Connect to the SQLite database
conn = sqlite3.connect('crime_data.db')
cursor = conn.cursor()

# Load the London boroughs GeoPackage file
boroughs_gdf = gpd.read_file('London_Boroughs.gpkg')
boroughs_gdf = boroughs_gdf.to_crs(epsg=4326)

# Add a new column 'borough' to the 'metropolitan-stop-and-search' table if it doesn't exist
try:
    cursor.execute('ALTER TABLE "metropolitan-stop-and-search" ADD COLUMN borough TEXT')
    conn.commit()
except sqlite3.OperationalError as e:
    if 'duplicate column name' in str(e):
        pass
    else:
        raise

# Fetch distinct years from the data
cursor.execute('SELECT DISTINCT strftime("%Y", Date) AS year FROM "metropolitan-stop-and-search"')
years = cursor.fetchall()

# Batch size for updates
batch_size = 50

# Process each year separately
for year in years:
    year = year[0]  # Extract year from tuple
    print(f"Processing data for year {year}")
    
    # Fetch rows for the current year
    cursor.execute('SELECT Date, Longitude, Latitude FROM "metropolitan-stop-and-search" WHERE strftime("%Y", Date) = ?', (year,))
    rows = cursor.fetchall()
    
    # Create a spatial index on the boroughs for faster lookups
    boroughs_sindex = boroughs_gdf.sindex
    
    # Prepare a list of updates
    updates = []
    
    # Process each row and prepare the update statement
    for row in tqdm(rows, desc=f"Processing rows for year {year}"):
        date, lon, lat = row
        if lon is not None and lat is not None:
            borough_name = get_borough(lon, lat, boroughs_gdf, boroughs_sindex)
            if borough_name:
                updates.append((borough_name, date))
                # When batch size is reached, execute the updates
                if len(updates) >= batch_size:
                    cursor.executemany('UPDATE "metropolitan-stop-and-search" SET borough = ? WHERE Date = ?', updates)
                    conn.commit()
                    updates.clear()
    
    # Execute any remaining updates
    if updates:
        cursor.executemany('UPDATE "metropolitan-stop-and-search" SET borough = ? WHERE Date = ?', updates)
        conn.commit()

# Close the connection
conn.close()


Processing data for year 2020


Processing rows for year 2020:   0%|          | 54/317505 [00:28<46:48:11,  1.88it/s]


KeyboardInterrupt: 

In [6]:
import sqlite3
import geopandas as gpd
from shapely.geometry import Point
from tqdm import tqdm

# Function to get borough name for a given point
def get_borough(lon, lat, boroughs_gdf, boroughs_sindex):
    point = Point(lon, lat)
    possible_matches_index = list(boroughs_sindex.intersection(point.bounds))
    possible_matches = boroughs_gdf.iloc[possible_matches_index]
    for _, borough in possible_matches.iterrows():
        if borough['geometry'].contains(point):
            return borough['name']  # Assuming 'name' is the column with the borough names
    return None

# Connect to the SQLite database
conn = sqlite3.connect('crime_data.db', timeout=10)
cursor = conn.cursor()

# Load the London boroughs GeoPackage file
boroughs_gdf = gpd.read_file('London_Boroughs.gpkg')
boroughs_gdf = boroughs_gdf.to_crs(epsg=4326)

# Add a new column 'borough' to the 'metropolitan-stop-and-search' table if it doesn't exist
try:
    cursor.execute('ALTER TABLE "metropolitan-stop-and-search" ADD COLUMN borough TEXT')
    conn.commit()
except sqlite3.OperationalError as e:
    if 'duplicate column name' in str(e):
        pass
    else:
        raise

# Fetch distinct years from the data
cursor.execute('SELECT DISTINCT strftime("%Y", Date) AS year FROM "metropolitan-stop-and-search"')
years = cursor.fetchall()

# Process each year separately
for year in years:
    year = year[0]  # Extract year from tuple
    print(f"Processing data for year {year}")
    
    # Fetch rows for the current year
    cursor.execute('SELECT Date, Longitude, Latitude FROM "metropolitan-stop-and-search" WHERE strftime("%Y", Date) = ?', (year,))
    rows = cursor.fetchall()
    
    # Create a spatial index on the boroughs for faster lookups
    boroughs_sindex = boroughs_gdf.sindex
    
    # Process each row and update the database immediately
    for row in tqdm(rows, desc=f"Processing rows for year {year}"):
        date, lon, lat = row
        if lon is not None and lat is not None:
            borough_name = get_borough(lon, lat, boroughs_gdf, boroughs_sindex)
            if borough_name:
                cursor.execute('UPDATE "metropolitan-stop-and-search" SET borough = ? WHERE Date = ?', (borough_name, date))
                conn.commit()

# Close the connection
conn.close()


Processing data for year 2020


Processing rows for year 2020:   0%|          | 0/317505 [00:11<?, ?it/s]


OperationalError: database is locked